In [3]:
import sys
sys.path.append('../tools')
from plot_tools import *
from map_tools import *
from params import *
from os.path import splitext

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
mapper = joblib.load('../tools/mapper.sav')
kde = joblib.load('../tools/kde_0_3.sav')
# curves = joblib.load('../tools/contours_kde.sav')
heatmap = joblib.load('../tools/heatmap_kde.sav')
curves = hv.operation.contours(heatmap, levels=[kde_th])

Set the injection type and injection set (made several rounds of injections with different parameters, the ones relevant now are all_sky_based and no_constraints)

In [25]:
# ccsn_paper = 'abdikamalov'
# ccsn_paper = 'andersen'
ccsn_paper = 'radice'

inj_set = ''
# inj_set = 'no_constraints'

data_path = join(features_path, 'segments/' + '/H1/injected/' + inj_set + '/ccsn/' + ccsn_paper)
files = [f for f in sorted(listdir(data_path)) if isfile(join(data_path, f)) and f.split('.')[-1] == 'hdf5']
print(len(files))

70


In [26]:
def parse_file_name(file, ccsn_paper):
    """Function to parse injection file name
    """
    if ccsn_paper == 'abdikamalov':
        ccsn_model = file.split('-')[2]
        D = float(splitext(join(data_path, file))[0].split('-')[-1])
    
    elif ccsn_paper == 'andersen':
        ccsn_model = '-'.join(file.split('-')[2:5])
        D = float(splitext(join(data_path, file))[0].split('-')[-1])
    
    elif ccsn_paper == 'radice':
        ccsn_model = file.split('-')[2]
        D = float(splitext(join(data_path, file))[0].split('-')[-1])
    
    return ccsn_model, D

In [27]:
x_examples, features_examples, y_examples, times_examples = load_train_examples(condition_method, model, features_path, data_path=features_path)
labels = np.unique(y_examples)

umap_examples = mapper.transform(features_examples)
tomap = {'ex': {'x': x_examples, 'y': y_examples, 'umap': umap_examples, 'times': times_examples}}

In [28]:
times_tot = []
ccsn_model = []
D = []
umap_tot = {'H': [], 'L': []}
y_hat_tot = {'H': [], 'L': []}
kde_scores = {'H': [], 'L': []}
devs = {'H': [], 'L': []}

num_points = 0

for num in range(len(files)):
    file = files[num]
    
    for detector in ['H', 'L']:
        data_path = join(features_path, 'segments/' + detector + '1/injected/' + inj_set + '/ccsn/' + ccsn_paper)
        
        with h5py.File(join(data_path, file), 'r') as f:
            umap = np.asarray(f['umap'])
            times = np.asarray(f['times'])
            y_hat = np.asarray(f['y_hat'])
            total_dev = np.asarray(f['total_dev'])
        
        pred_idx = np.argmax(y_hat, axis=1)
        umap_tot[detector].extend(umap)
        y_hat_tot[detector].extend(labels[pred_idx])
        kde_scores[detector].extend(kde.score_samples(umap))
        devs[detector].extend(total_dev)
        if detector == 'H':
            times_tot.extend(times)
        
    num_points += len(times)
    parse_res = parse_file_name(file, ccsn_paper)
    ccsn_model.extend([parse_res[0]] * len(times))
    D.extend([parse_res[1]] * len(times))
    
for key in umap_tot.keys():
    umap_tot[key] = np.asarray(umap_tot[key])
    
print(num_points) #, len(times_tot), len(inj_type_tot), len(kde_scores['H']), len(devs['L']))
print(umap_tot['H'].shape, len(y_hat_tot['H']))

df = pd.DataFrame({'time': times_tot, 'model': ccsn_model, 'D': D, 'H_kde': kde_scores['H'], 
                   'L_kde': kde_scores['L'], 'H_dev': devs['H'], 'L_dev': devs['L'], 
                   'H_pred': y_hat_tot['H'], 'L_pred': y_hat_tot['L']})
display(df.head())

1050
(1050, 2) 1050


,time,model,D,H_kde,L_kde,H_dev,L_dev,H_pred,L_pred
0,1.127032e+09,s10,0.2,-5.015149,-4.994838,728.952987,710.718006,Wandering_Line,Wandering_Line
1,1.136360e+09,s10,0.2,-5.822944,-5.519241,1382.063340,520.155928,None_of_the_Above,None_of_the_Above
2,1.126092e+09,s10,0.2,-5.128578,-7.712664,424.615736,457.854188,Whistle,Whistle
3,1.126581e+09,s10,0.2,-7.172629,-5.053991,1795.055882,84.240559,Wandering_Line,Whistle
4,1.126651e+09,s10,0.2,-7.626987,-7.491241,1163.806286,899.062470,Whistle,Whistle


In [29]:
umap = np.concatenate((umap_tot['H'], umap_tot['L']))
y_hat = np.concatenate((y_hat_tot['H'], y_hat_tot['L']))
times = np.concatenate((times_tot, times_tot))
print(umap.shape)
# print(np.asarray(len(list(umap_tot['H']))).shape)
title = ccsn_paper + ' injections'
layout = interactive_plot(umap, y_hat, times, title=title) * curves
layout

(2100, 2)


:Overlay
   .Points.I   :Points   [feature_1,feature_2]   (times,y,marker,size,index)
   .Contours.I :Contours   [x,y]   (z)

In [30]:
dev_th = gram_th
print(kde_th, dev_th)

-11.5 5100


In [31]:
H_kde_count = []
L_kde_count = []
both_kde_count = []
H_dev_count = []
L_dev_count = []
both_dev_count = []
both_nota_count = []
single_nota_count = []
both_chirp_count = []
single_chirp_count = []

models = []
D_tot = []

for ccsn_model in np.unique(df['model']):
    for D in np.unique(df['D']):
        models.append(ccsn_model)
        D_tot.append(D)
        H_kde_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & (df['H_kde'] < kde_th)))
        L_kde_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & (df['L_kde'] < kde_th)))
        both_kde_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & (df['H_kde'] < kde_th) & (df['L_kde'] < kde_th)))
        H_dev_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & (df['H_dev'] > dev_th)))
        L_dev_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & (df['L_dev'] > dev_th)))
        both_dev_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & (df['H_dev'] > dev_th) & (df['L_dev'] > dev_th)))
        both_nota_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & (df['H_pred'] == 'None_of_the_Above') & \
                                   (df['L_pred'] == 'None_of_the_Above')))
        single_nota_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & (((df['H_pred'] == 'None_of_the_Above') & \
                                   (df['L_pred'] != 'No_Glitch')) | ((df['H_pred'] != 'No_Glitch') & (df['L_pred'] == 'None_of_the_Above')))))
        both_chirp_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & (df['H_pred'] == 'Chirp') & (df['L_pred'] == 'Chirp')))
        single_chirp_count.append(sum((df['model'] == ccsn_model) & (df['D'] == D) & ((df['H_pred'] == 'Chirp') | (df['L_pred'] == 'Chirp'))))

stats = pd.DataFrame({'model': models, 'D': D_tot})
stats['H_kde_count'] = H_kde_count
stats['L_kde_count'] = L_kde_count
stats['both_kde_count'] = both_kde_count
stats['H_dev_count'] = H_dev_count
stats['L_dev_count'] = L_dev_count
stats['both_dev_count'] = both_dev_count
stats['both_nota_count'] = both_nota_count
stats['single_nota_count'] = single_nota_count
stats['both_chirp_count'] = both_chirp_count
stats['single_chirp_count'] = single_chirp_count

In [32]:
idx = pd.Series([False] * len(stats))
for col_num in [2, 3, 5, 6, 8]: #, 9, 10]:
    idx |= (stats[stats.columns[col_num]] != 0) & (stats['D'] >= 0.2)
#     idx |= (stats[stats.columns[col_num]] >= 5)
#     idx |= (stats[stats.columns[col_num]] >= 8) & (stats['D'] >= 0.2)

display(stats[idx])

,model,D,H_kde_count,L_kde_count,both_kde_count,H_dev_count,L_dev_count,both_dev_count,both_nota_count,single_nota_count,both_chirp_count,single_chirp_count
0,s10,0.2,0,0,0,1,0,0,1,2,0,0
1,s10,0.5,0,1,0,0,0,0,0,0,0,0
7,s11,0.2,0,0,0,0,1,0,1,5,0,0
14,s12,0.2,0,0,0,0,0,0,5,6,0,0
21,s13,0.2,0,0,0,2,0,0,5,9,0,0
22,s13,0.5,0,0,0,1,0,0,0,0,0,0
28,s14,0.2,1,0,0,1,0,0,4,5,0,0
35,s15,0.2,0,0,0,0,0,0,4,7,0,0
42,s19,0.2,0,0,0,0,0,0,7,8,0,0
49,s25,0.2,0,0,0,0,1,0,7,10,0,0
